# Basics of Neural Network     

## Binary Classification
## Logistic Regression
### Sigmoid function
$$ \hat{y}=P(y=1 | x)=\sigma(w^Tx + b),  0\leq \hat{y}\leq 1$$

### Cost function
For one sample:  
Square Error:  
$$Loss(\hat{y},y)=\frac{1}{2}(\hat{y}- y)^2$$
It makes GD not work well.  
**Information Cross Entropy**:(A Better choice)  
$$Loss(\hat{y},y)=-(y\log{\hat{y}}+(1-y)\log{(1-\hat{y})})$$
For the dataset, we calculate the mean of the error:  
$$J(w,b)=\frac{1}{m}\sum_{i}^m Loss(\hat{y}^{(i)},y^{(i)}) $$

### Gradient Descent
We want to **minimize J(w,b) **   
Algorithm : seen in Zhouzhihua ML book  
Derivative in a Computation Graph:  chain rule  
$$ z = w^Tx + b,$$
$$\hat y=\sigma {(z)}=\sigma{ (w^Tx + b)}$$
You can deduce that:(if using cross entropy)    
$$\frac{\partial L}{\partial z} = \hat y-y$$
Hence  
$$\frac{\partial L}{\partial w_i}=x_i\frac{\partial L}{\partial z}\quad $$

$$\frac{\partial L}{\partial b}=\frac{\partial L}{\partial z}\quad $$
$$w_i=w_i-\alpha \frac{\partial L}{\partial w_i}$$

## For m samples:
### Vectorization: get rid of 'for' loop in code    
In deep learning, you deal with very large datasets. Hence, a non-computationally-optimal function can become a huge bottleneck in your algorithm and can result in a model that takes ages to run. To make sure that your code is computationally efficient, you will use vectorization. For example, try to tell the difference between the following implementations of the dot/outer/elementwise product.

In [6]:
import numpy as np

import time
a=np.random.random(1000000)
b=np.random.random(1000000)

#np.dot(w,x)  #realize $w^Tx + b$

tic =time.time()
z = np.dot(a,b)
toc=time.time()
print(z)
print('Vectorization version:'+ str(1000*(toc-tic))+' ms')

249951.474113
Vectorization version:2.001047134399414 ms


We can compare with the loop method , the vectorization is much faster

And we go on to focus on m-sample question  
we assume $\frac{\partial L}{\partial z} = \hat y-y = \mathrm{d}z$  in one sample  
so we build the matrix that each column vector is one-sample derivative of z
$$dz = [dz^{(1)},dz^{(2)},...dz^{(m)}]$$
X is the imput matrix composed with column vectors $x^{(1)},x^{(2)},...$  
 $$dw=\frac{\partial J}{\partial w}=1/m * X (dz)^T$$
 $$db=\frac{\partial J}{\partial b}=1/m * np.sum(dz)$$
the result is a 
And you still need a loop to do GD on the same dataset and the loop is inevitable

### Broadcast in python

In [24]:
a=np.random.randn(1,4)
a

array([[ 0.19161645,  0.1538191 ,  0.70168727,  0.19568387]])

In [25]:
a+100

array([[ 100.19161645,  100.1538191 ,  100.70168727,  100.19568387]])

In [26]:
b= a.reshape((2,2))
b

array([[ 0.19161645,  0.1538191 ],
       [ 0.70168727,  0.19568387]])

In [27]:
b.sum(axis=0)

array([ 0.89330372,  0.34950298])

In [28]:
b+np.array([[100],[200]])

array([[ 100.19161645,  100.1538191 ],
       [ 200.70168727,  200.19568387]])

## Neural Networks

For layer k  ,the weights we assign $\mathbf{W^{[k]}}$  
the activation input is $z^{[k]}$   ,partial derivative of $z^{[k]}$ is $dz^{[k]}$  
the activation output is $A^{[k]}=[a^{[k]}_0,a^{[k]}_1,a^{[k]}_2...]^T$ , and we assume that $ A^{[0]}= X $ 
layer k has $n^{[k]}$ nodes  
Note : the number of layers of the network : normally donot count the input layer(layer 0)

The computation of the neural network is clear, and let's focus on the vectorization implementation  
m samples computation using matrix computing  
For example
$$W^{[1]}X+b^{[1]}=z^{[1]} $$ for the following layer is 
$$W^{[k]}A^{[k-1]}+b^{[k]}=z^{[k]}$$
X is composed with column vector $x^{(1)},x^{(2)},...$
$$A^{[k]}=[a^{[k](1)},a^{[k](2)},a^{[k](3)},...,a^{[k](m)}],shape=(n^{[k]},m)$$
Hence  
<font color=red size=5>
$$dW^{[k]}=\frac{\partial J}{\partial W^{[k]}}=\frac{1}{m}  dz^{[k]}(A^{[k-1]})^T $$
</font>

### Activation function
non-linear function  
#### Most common activation func :   
- sigmoid (between 0 and 1)
- tanh (between -1 and 1) the averge output is 0 (benefit,superer than sigmoid)
- Relu (Rectified linear unit) $ a = max(0,z)$
- leaky Relu (based on Relu ,when input is negative, the derivative is no longer 0, but a very small positive value) $a=max(0.01z,z)$  

if don't know which is better , can evaluate them on a holdout validation set

#### derivative of activation func

func: g  
- sigmoid: $a=g(z)$,$g'(z)=a(1-a)$  
- tanh: $a=g(z)$,$g'(z)=1-a^2$  
- Relu: $g'(z)=0(if\quad z<0)\quad=1(if \quad z>0)$  

### BP algorithm in NN

For two layer NN,  
We assume that $n^{[0]}$ is the number of input x feat ,$n^{[1]}$ is the number of units in hidden layer,$n^{[2]}=1$ is the number of outputs  
So the shape of $W^{[1]}$ is $$(n^{[1]},n^{[0]})$$
the shape of $b^{[1]}$ is $$(n^{[1]},1)$$
the shape of X is $$(n^{[0]},m)$$
the shape of $W^{[2]}$ is $$(n^{[2]},n^{[1]})$$
the shape of $b^{[2]}$ is $$(n^{[2]},1)$$
Hence the shape of output $A^{[2]}$ is $$(n^{[2]},m)$$
Forward Propagation: shown above  
Back Propagation: 
$$dz^{[2]}=A^{[2]}-y, A^{[2]}=\hat y $$
$$y=[y^{(1)},y^{(2)},y^{(3)},...y^{(m)}]\quad shape:(n^{[2]},m)$$

$$db^{[k]}=\frac{1}{m}np.sum(dz^{[k]},axis=1,keepdims=True)$$
we use (keepdims = True) to make sure that A.shape is (dim,1) and not (dim, ). It makes our code more rigorous
<font color=red size=5>
$$ dz^{[1]}=\frac{\partial J}{\partial z^{[2]}}\frac{\partial z^{[2]}}{\partial A^{[1]}}\frac{\partial A^{[1]}}{\partial z^{[1]}}\\ =(W^{[2]})^Tdz^{[2]} * g'(z^{[1]})$$
</font>
<center> it can be seen as $dz^{[k]}=dA^{[k]}\cdot g'(z^{[k]})$  
$dA^{[k-1]}$ can be deduced from $dz^{[k]}$ and $dz^{[k]}$ can be deduced from $dA^{[k]}$, and this realizes the back prop   
the * is element-wise product ,cus the shape of operandant are both $(n^{[1]},m)$</center>  


Now we can calculate the dz,db to derive dW using the formulas derived above using red color

### Initialize the weight randomly

Disadvantage of initialize the weight to zero  


W1=np.random.randn(n[1],n[0]) * 0.01  
initialize it to a very small random value  
Why??  
If the weight is large , the activation value is very large and it falls on the flat part of the activation func ,leading to the small gradient, and the gradient descent will be very slow

## Deep NN

### Forward propagation vectorization:
it's ok to implement a loop to compute z and A  
### Check the matrix dimension
For layer k:  
Shape of $W^{[k]}$ is $$(n^{[k]},n^{[k-1]})$$
Shape of $b^{[k]}$ is $$(n^{[k]},1)$$
the dimension of dw should be the same as the dimension of W, as well as db to b  
z and A should have the same dimension  $$(n^{[k]},m)$$
### Why we build a 'deep' network
have the earlier layers learn these low-level simpler features and in the deep layers we put together the simple features that detected  
In curcuit theory, some logistic gates'function can compute with a relatively small but deep network.
### Building blocks of deep NN
It will be very useful to store the $z^{[k]}$  for both forward and back prop.(include this on cache )
grads and $A^{[k]}$  
<img src="images/forward_and_back_prop.png" style="width:600px;height:300px;">
the blocks in back propagation will output the grads--dw and db  
### Implement the blocks of back propagation vectorization:
Forward prop for layer k:  
Input : $A^{[k-1]}$  
Output: $A^{[k]}$  cache: $z^{[k]}$ (it can be calculated from $W^{[k]},b^{[k]}$  
from k=1, A[0]=X is fed. And implement the forward prop ,cache z . 

Back prop for layer k:  
Input : $dA^{[k]}$  
Output: $dA^{[k-1]},dW^{[k]},db^{[k]}$  
<font color=blue size=5>
$$dz^{[k]}=dA^{[k]}\cdot g'(z^{[k]})$$

$$dW^{[k]}=\frac{\partial J}{\partial W^{[k]}}=\frac{1}{m}  dz^{[k]}(A^{[k-1]})^T $$

$$db^{[k]}=\frac{1}{m}np.sum(dz^{[k]},axis=1,keepdims=True)$$

$$dA^{[k-1]}=(W^{[k]})^Tdz^{[k]}$$
</font>

## Final Outline of the deep NN
<img src="images/final outline.png" style="width:600px;height:500px;">

## Organize hyperparameters well

Hyperparameters :   
1. Learning rate  
2. Number of iterations   
3. Number of hidden layers  
4. Hidden units of each hidden layer  
5. Choice of activation function  
These control W and b ,so call them hyperparameter 